In [ ]:
pip install sentence-transformers langchain faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 74.1 MB/s eta 0:00:00


In [ ]:
pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 59.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.8 MB/s eta 0:00:00


In [ ]:
pip install pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.5/310.5 kB 9.9 MB/s eta 0:00:00


In [ ]:
pdf_path = "/content/MOSDAC.pdf"   # change name if needed

from langchain_community.document_loaders import PyPDFLoader
from langchain.schema import Document

# STEP 1: Load single PDF
def load_pdf(file_path):
    loader = PyPDFLoader(file_path)
    docs = loader.load()
    return docs

documents = load_pdf(pdf_path)

print(f"✅ Loaded {len(documents)} pages from PDF.")

✅ Loaded 95 pages from PDF.


In [ ]:
from langchain_community.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from langchain.schema import Document

In [ ]:
# Load PDF
pdf_path = "/content/MOSDAC.pdf"
loader = PyPDFLoader(pdf_path)
documents = loader.load()
print(f"✅ Loaded {len(documents)} pages from PDF.")

# Split into chunks
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=100
)
chunks = text_splitter.split_documents(documents)
print(f"✅ Chunked into {len(chunks)} smaller pieces.")

# Create embeddings
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Create FAISS vectorstore
vectorstore = FAISS.from_documents(chunks, embedding_model)

# Save FAISS index
vectorstore.save_local("mosdac_faiss_index")
print("✅ FAISS index saved as 'mosdac_faiss_index'")

✅ Loaded 95 pages from PDF.
✅ Chunked into 397 smaller pieces.


/tmp/ipython-input-2873703803.py:16: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public mode

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

✅ FAISS index saved as 'mosdac_faiss_index'


In [ ]:
from langchain_community.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings

# Reload embeddings (must match the one used when saving FAISS)
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Load FAISS index
vectorstore = FAISS.load_local(
    "mosdac_faiss_index",
    embedding_model,
    allow_dangerous_deserialization=True
)

# Example query
query = "How can I register in MOSDAC?"
results = vectorstore.similarity_search(query, k=3)

print("🔍 Retrieved Contexts:")
for i, doc in enumerate(results, 1):
    print(f"\n--- Result {i} ---")
    print(doc.page_content[:500])  # show first 500 chars

🔍 Retrieved Contexts:

--- Result 1 ---
copyright holders concerned. 
Frequently Asked Questions 
What is MOSDAC? 
MOSDAC is the short form of Meteorological an d Oceanographic Satellite Data Archival Center. 
It is a ISRO data portal which provides data through its web based service https://mosdac.gov.in 
 
How to be a registered user of MOSDAC? 
There is "SignUp" form available on MOSDAC portal. Pl fill up the form and submit. You will 
be intimated through e-mail about the approval

--- Result 2 ---
Login with your user name and password. select 'Change Profile'.  Your Profile page will be 
displayed. Select 'Password' from the left panel of Profile Page. Provide old and new 
password and click save. Your password will be changed. 
I don't have username and password of MOSDAC. Can I download data?  
Anonymous users (without username and password) can download only open data. In order 
to download all data from MOSDAC, you need to register on MOSDAC site. There is "SignUp"

--- Resul

In [ ]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, classification_report

# Example ground-truth and predicted intents (replace with your real data)
true_intents = ['get_weather', 'get_satellite', 'download_data', 'get_weather', 'faq_lookup']
predicted_intents = ['get_weather', 'faq_lookup', 'download_data', 'faq_lookup', 'faq_lookup']

# Accuracy
intent_accuracy = accuracy_score(true_intents, predicted_intents)
print(f"✅ Intent Recognition Accuracy: {intent_accuracy:.2f}")

# Precision, Recall, F1 per class
precision, recall, f1, _ = precision_recall_fscore_support(true_intents, predicted_intents, average='weighted')
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1-score: {f1:.2f}")

✅ Intent Recognition Accuracy: 0.60
Precision: 0.67
Recall: 0.60
F1-score: 0.57


/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
from sklearn.metrics import precision_recall_fscore_support, classification_report

# Example entity recognition evaluation
true_entities = [['rainfall'], ['INSAT', 'download'], ['cloud_data']]
predicted_entities = [['rainfall'], ['INSAT'], ['cloud_data']]

# Flatten for evaluation
true_flat = [ent for ents in true_entities for ent in ents]
pred_flat = [ent for ents in predicted_entities for ent in ents]

# Align lengths (pad predicted if needed, or truncate)
min_len = min(len(true_flat), len(pred_flat))
true_flat = true_flat[:min_len]
pred_flat = pred_flat[:min_len]

# Evaluate
precision, recall, f1, _ = precision_recall_fscore_support(true_flat, pred_flat, average='micro')
print(f"✅ Entity Recognition - Precision: {precision:.2f}, Recall: {recall:.2f}, F1 Score: {f1:.2f}")

✅ Entity Recognition - Precision: 0.67, Recall: 0.67, F1 Score: 0.67
